# Catchment Boundaries

Notebook to create the files `CAMELS_DE_catchment_boundaries.geopackage` and `CAMELS_DE_catchment_boundaries.shp`.  
As well as `CAMELS_DE_gauging_stations.geopackage` and `CAMELS_DE_gauging_stations.shp`.  

We always use `EPSG:3035` as the coordinate reference system.

In [1]:
from glob import glob
import os
import pandas as pd
import geopandas as gpd

from camelsp import Station

/home/alexd/miniconda3/lib/python3.11/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.3-CAPI-1.17.3) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_140275/1072211159.py:5: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS dir

In [2]:
# get camels_ids from hydromet timeseries
camels_ids = [camels_id.split("_")[-1].split(".csv")[0] for camels_id in glob("../output_data/camels_de/timeseries/*.csv")]

# sort camels_ids
camels_ids = sorted(camels_ids)

print(f"Total number of stations in CAMELS-DE v1: {len(camels_ids)}")

Total number of stations in CAMELS-DE v1: 1460


## Save catchment boundaries

Get all the catchment boundaries from the CAMELS-DE dataset and save them in a geopackage and shapefile.

In [3]:
catchments = []

for camels_id in camels_ids:
    # initialize station
    s = Station(camels_id)

    # get MERIT Hydro catchment
    catchment = s.get_catchment("merit_hydro")

    # from epsg:4326 to epsg:3035
    catchment = catchment.to_crs(epsg=3035)

    # add catchment to catchments
    catchments.append(catchment)

# concatenate all catchments into a single GeoDataFrame
catchments = gpd.pd.concat(catchments, ignore_index=True)
catchments.crs = "EPSG:3035"

# make directory if not exists
os.makedirs("../output_data/camels_de/CAMELS_DE_catchment_boundaries", exist_ok=True)
os.makedirs("../output_data/camels_de/CAMELS_DE_catchment_boundaries/catchments", exist_ok=True)

# save catchments
catchments.to_file("../output_data/camels_de/CAMELS_DE_catchment_boundaries/catchments/CAMELS_DE_catchments.shp")
catchments.to_file("../output_data/camels_de/CAMELS_DE_catchment_boundaries/catchments/CAMELS_DE_catchments.gpkg", driver="GPKG")

100%|██████████| 1460/1460 [00:37<00:00, 38.52it/s]
/tmp/ipykernel_140275/2167409807.py:25: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  catchments.to_file("../output_data/camels_de/CAMELS_DE_catchment_boundaries/catchments/catchments.shp")


## Save gauging station locations

In [4]:
# read station locations from topographic attributes
df = pd.read_csv("../output_data/camels_de/CAMELS_DE_topographic_attributes.csv")

# convert to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["gauge_easting"], df["gauge_northing"]))
gdf.crs = "EPSG:3035"

# make directory if not exists
os.makedirs("../output_data/camels_de/CAMELS_DE_catchment_boundaries/stations", exist_ok=True)

# save station locations
gdf.to_file("../output_data/camels_de/CAMELS_DE_catchment_boundaries/stations/CAMELS_DE_gauging_stations.shp")
gdf.to_file("../output_data/camels_de/CAMELS_DE_catchment_boundaries/stations/CAMELS_DE_gauging_stations.gpkg", driver="GPKG")

/tmp/ipykernel_140275/1604251584.py:12: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file("../output_data/camels_de/CAMELS_DE_catchment_boundaries/stations/stations.shp")
